In [ ]:
import torchvision.models as models
import torch

import numpy as np
import matplotlib.pyplot as plt

from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem

In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats

In [ ]:
mats = get_mobilenet_target_mats()

In [ ]:
mats[0]

Calculate the cost of the stages

 

$n$ is the state dimenstion

$m$ is the input dimension

$p$ is the output dimension

causal:
$$m_{k} n_{k+1} + m_{k} p_{k} + n_{k} n_{k+1} + n_{k} p_{k}$$

anticausal:
$$m_{k} n_{k-1} + m_{k} p_{k} + n_{k} n_{k-1} + n_{k} p_{k}$$


In [ ]:
def cost_sys(system,include_D=True):
    c = 0
    if include_D:
        for stage in system.stages:
            c += stage.A_matrix.size + stage.B_matrix.size +stage.C_matrix.size +stage.D_matrix.size
    else:
        for stage in system.stages:
            c += stage.A_matrix.size + stage.B_matrix.size +stage.C_matrix.size
    return c


def cost(system,include_D_anticausal=False):
    if type(system) == MixedSystem:
        return cost_sys(system.causal_system)+ cost_sys(system.anticausal_system,include_D =include_D_anticausal)
    else:
        cost_sys(system)

# Test some changes in properties

We take a section out of the Matrix to make the exploration easier. 
We change the sizes of the input and output dimention. 

In [ ]:
#some preperations
matrix = mats[0][:100,:100]
dims_a = np.ones(2,dtype=int)*50
dims_b = np.ones(20,dtype=int)*5

In [ ]:
T = ToeplitzOperator(matrix, dims_a, dims_a)
S = SystemIdentificationSVD(T,epsilon=1e-16)

system = MixedSystem(S)

In [ ]:
np.linalg.norm(system.to_matrix()-matrix)

In [ ]:
plt.imshow(matrix)

In [ ]:
print(system.causal_system.dims_state)
print(system.anticausal_system.dims_state)
print("Cost:",cost(system))

In [ ]:
ks = [2,5,10,25,50]
costs = np.zeros_like(ks)
err = np.zeros_like(ks,dtype=np.float64)


for i,k in enumerate(ks):
    dims = np.ones(k,dtype=int)*(100//k)
    T = ToeplitzOperator(matrix, dims, dims)
    S = SystemIdentificationSVD(T,epsilon=1e-4)

    system = MixedSystem(S)
    err[i] = np.linalg.norm(system.to_matrix()-matrix)
    costs[i] = cost(system)

    

plt.subplot(1,2,1)
plt.scatter(ks,costs)
plt.ylabel("cost")
plt.xlabel("number of stages")
plt.subplot(1,2,2)
plt.scatter(ks,err)
plt.ylabel("Frobenius norm")
plt.xlabel("number of stages")

In [ ]:
costs

In [ ]:
ks

In [ ]:
system

In [ ]:
system.causal_system.stages[10].A_matrix.size

In [ ]:
system.anticausal_system.dims_state